In [ ]:
!pip install lifelines --quiet

import pandas as pd
import numpy as np
from lifelines import CoxPHFitter


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.5 MB/s eta 0:00:00


In [ ]:
patients_path = "/content/ENGR-E 483_583_Patients_Data (1).xlsx"
history_path  = "/content/ENGR-E 483_583_Medical_History_final (2).xlsx"
events_path   = "/content/ENGR-E 483_583_Health_Events_cleaned (1).xlsx"
env_path      = "/content/statewise_data_health_env_cleaned (1).xlsx"

# Patients
patients = pd.read_excel(
    patients_path,
    usecols=[
        "Medcal_Record_Number", "Age", "Gender", "Ethnicity", "BMI",
        "Smoking_Status", "Alcohol_Consumption",
        "Physical_Activity_Level", "Medicaid_eligible",
        "Living_Alone", "Dietary_Preference", "Opioid_Addiction",
        "City", "State"
    ]
)

# History (for index date)
history = pd.read_excel(
    history_path,
    usecols=["Medcal_Record_Number", "Diagnosed_date", "Diagnosis_Year"],
    parse_dates=["Diagnosed_date"]
)

# Events
events = pd.read_excel(
    events_path,
    usecols=["Medcal_Record_Number", "Event_Type", "Event_Date", "Outcome"],
    parse_dates=["Event_Date"]
)

# Environment
env = pd.read_excel(
    env_path,
    usecols=[
        "State", "Year", "Pollution_Index",
        "Adjusted_Pollution", "Env_Health_Burden",
        "Access_Index", "Health_Access_Category"
    ]
)

print("patients:", patients.shape)
print("history:", history.shape)
print("events:", events.shape)
print("env:", env.shape)


patients: (100000, 14)
history: (213234, 3)
events: (29855, 4)
env: (3056, 7)


In [ ]:
idx = (
    history
    .sort_values(["Medcal_Record_Number", "Diagnosed_date"])
    .groupby("Medcal_Record_Number", as_index=False)
    .first()[["Medcal_Record_Number", "Diagnosed_date", "Diagnosis_Year"]]
    .rename(columns={"Diagnosed_date": "index_date"})
)

idx.head()


,Medcal_Record_Number,index_date,Diagnosis_Year
0,P000013831,1964-09-20,1964
1,P000014481,2010-06-01,2010
2,P000052349,1983-01-24,1983
3,P000052842,1987-09-20,1987
4,P000064164,1982-08-14,1982


In [ ]:
# lowercase helpers
events["Event_Type_low"] = events["Event_Type"].str.lower()
events["Outcome_low"] = events["Outcome"].str.lower()

# rows that indicate death
death_rows = events[
    (events["Event_Type_low"] == "death") |
    (events["Outcome_low"] == "fatal")
].copy()

# earliest death per patient
death_per_patient = (
    death_rows
    .groupby("Medcal_Record_Number", as_index=False)["Event_Date"]
    .min()
    .rename(columns={"Event_Date": "death_date"})
)

# last event per patient (for censoring if no death)
last_event = (
    events
    .groupby("Medcal_Record_Number", as_index=False)["Event_Date"]
    .max()
    .rename(columns={"Event_Date": "last_event_date"})
)

# merge with index_date
surv_death = (
    idx
    .merge(death_per_patient, on="Medcal_Record_Number", how="left")
    .merge(last_event, on="Medcal_Record_Number", how="left")
)

# event indicator for death
surv_death["event_death"] = np.where(surv_death["death_date"].notna(), 1, 0)

# if no last_event_date, use index_date (at least 1 day follow-up)
surv_death["last_event_date"] = surv_death["last_event_date"].fillna(surv_death["index_date"])

# stop date = death_date if exists, else last_event_date
surv_death["stop_date_death"] = np.where(
    surv_death["death_date"].notna(),
    surv_death["death_date"],
    surv_death["last_event_date"]
)

# duration in days to death/censoring
surv_death["duration_days_death"] = (
    (pd.to_datetime(surv_death["stop_date_death"]) - surv_death["index_date"])
    .dt.days
)

surv_death["duration_days_death"] = (
    surv_death["duration_days_death"]
    .fillna(1)
    .clip(lower=1)
)

surv_death.head()


,Medcal_Record_Number,index_date,Diagnosis_Year,death_date,last_event_date,event_death,stop_date_death,duration_days_death
0,P000013831,1964-09-20,1964,NaT,1964-09-20,0,1964-09-20,1
1,P000014481,2010-06-01,2010,NaT,2010-06-01,0,2010-06-01,1
2,P000052349,1983-01-24,1983,NaT,1983-01-24,0,1983-01-24,1
3,P000052842,1987-09-20,1987,NaT,1987-09-20,0,1987-09-20,1
4,P000064164,1982-08-14,1982,NaT,1982-08-14,0,1982-08-14,1


In [ ]:
# rows that indicate recovery
rec_rows = events[events["Outcome_low"] == "recovered"].copy()

# earliest recovery per patient
recovery_per_patient = (
    rec_rows
    .groupby("Medcal_Record_Number", as_index=False)["Event_Date"]
    .min()
    .rename(columns={"Event_Date": "recovery_date"})
)

# last event date (already computed as last_event)
surv_rec = (
    idx
    .merge(recovery_per_patient, on="Medcal_Record_Number", how="left")
    .merge(last_event, on="Medcal_Record_Number", how="left")
)

# event indicator for recovery
surv_rec["event_rec"] = np.where(surv_rec["recovery_date"].notna(), 1, 0)

# at least index_date as last_event_date if missing
surv_rec["last_event_date"] = surv_rec["last_event_date"].fillna(surv_rec["index_date"])

# stop date for recovery model
surv_rec["stop_date_rec"] = np.where(
    surv_rec["recovery_date"].notna(),
    surv_rec["recovery_date"],
    surv_rec["last_event_date"]
)

# duration in days to recovery/censor
surv_rec["duration_days_rec"] = (
    (pd.to_datetime(surv_rec["stop_date_rec"]) - surv_rec["index_date"])
    .dt.days
)

surv_rec["duration_days_rec"] = (
    surv_rec["duration_days_rec"]
    .fillna(1)
    .clip(lower=1)
)

surv_rec.head()


,Medcal_Record_Number,index_date,Diagnosis_Year,recovery_date,last_event_date,event_rec,stop_date_rec,duration_days_rec
0,P000013831,1964-09-20,1964,NaT,1964-09-20,0,1964-09-20,1
1,P000014481,2010-06-01,2010,NaT,2010-06-01,0,2010-06-01,1
2,P000052349,1983-01-24,1983,NaT,1983-01-24,0,1983-01-24,1
3,P000052842,1987-09-20,1987,NaT,1987-09-20,0,1987-09-20,1
4,P000064164,1982-08-14,1982,NaT,1982-08-14,0,1982-08-14,1


In [ ]:
# Merge death survival info with patients + env
death_df = (
    patients
    .merge(
        surv_death[["Medcal_Record_Number", "index_date", "Diagnosis_Year",
                    "duration_days_death", "event_death"]],
        on="Medcal_Record_Number", how="inner"
    )
    .merge(
        env,
        left_on=["State", "Diagnosis_Year"],
        right_on=["State", "Year"],
        how="left"
    )
    .drop(columns=["Year"])
)

# Merge recovery survival info
rec_df = (
    patients
    .merge(
        surv_rec[["Medcal_Record_Number", "index_date", "Diagnosis_Year",
                  "duration_days_rec", "event_rec"]],
        on="Medcal_Record_Number", how="inner"
    )
    .merge(
        env,
        left_on=["State", "Diagnosis_Year"],
        right_on=["State", "Year"],
        how="left"
    )
    .drop(columns=["Year"])
)

# Convert booleans to ints
for col in ["Medicaid_eligible", "Living_Alone", "Opioid_Addiction"]:
    death_df[col] = death_df[col].astype(int)
    rec_df[col]   = rec_df[col].astype(int)

print("death_df:", death_df.shape)
print("rec_df:", rec_df.shape)


death_df: (107274, 23)
rec_df: (107274, 23)


In [ ]:
N = 5000  # adjust up/down depending on Colab memory

if len(death_df) > N:
    death_df_s = death_df.sample(N, random_state=42)
else:
    death_df_s = death_df.copy()

if len(rec_df) > N:
    rec_df_s = rec_df.sample(N, random_state=42)
else:
    rec_df_s = rec_df.copy()

print("Using rows (death):", death_df_s.shape[0])
print("Using rows (rec):  ", rec_df_s.shape[0])


Using rows (death): 5000
Using rows (rec):   5000


In [ ]:
feature_cols = [
    "Age", "BMI",
    "Medicaid_eligible", "Living_Alone", "Opioid_Addiction",
    "Pollution_Index", "Adjusted_Pollution",
    "Env_Health_Burden", "Access_Index",
    "Gender", "Ethnicity", "Smoking_Status",
    "Alcohol_Consumption", "Physical_Activity_Level",
    "Dietary_Preference", "State", "Health_Access_Category"
]

num_cols = [
    "Age", "BMI",
    "Medicaid_eligible", "Living_Alone", "Opioid_Addiction",
    "Pollution_Index", "Adjusted_Pollution",
    "Env_Health_Burden", "Access_Index"
]

cat_cols = [
    "Gender", "Ethnicity", "Smoking_Status",
    "Alcohol_Consumption", "Physical_Activity_Level",
    "Dietary_Preference", "State", "Health_Access_Category"
]

def prepare_survival_df(df, duration_col, event_col):
    data = df[["Medcal_Record_Number", duration_col, event_col] + feature_cols].copy()

    # fill numeric
    for c in num_cols:
        data[c] = data[c].astype(float)
        data[c] = data[c].fillna(data[c].median())

    # fill categorical
    for c in cat_cols:
        data[c] = data[c].fillna("Unknown")

    data[duration_col] = data[duration_col].fillna(data[duration_col].median())
    data[event_col] = data[event_col].fillna(0).astype(int)

    # dummies
    data_dum = pd.get_dummies(data, columns=cat_cols, drop_first=True)
    data_dum = data_dum.replace([np.inf, -np.inf], np.nan)
    data_dum = data_dum.dropna()

    # rename duration+event for lifelines
    data_dum = data_dum.rename(columns={duration_col: "T", event_col: "E"})

    return data_dum

death_data = prepare_survival_df(death_df_s, "duration_days_death", "event_death")
rec_data   = prepare_survival_df(rec_df_s,   "duration_days_rec",   "event_rec")

print("death_data shape:", death_data.shape)
print("rec_data shape:", rec_data.shape)


death_data shape: (5000, 40)
rec_data shape: (5000, 40)


In [ ]:
# Death Cox model
cph_death = CoxPHFitter()
cph_death.fit(
    death_data.drop(columns=["Medcal_Record_Number"]),
    duration_col="T",
    event_col="E"
)

# Recovery Cox model
cph_rec = CoxPHFitter()
cph_rec.fit(
    rec_data.drop(columns=["Medcal_Record_Number"]),
    duration_col="T",
    event_col="E"
)


/usr/local/lib/python3.12/dist-packages/lifelines/utils/__init__.py:1120: ConvergenceWarning: Column State_Oregon have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['E'].astype(bool)
>>> print(df.loc[events, 'State_Oregon'].var())
>>> print(df.loc[~events, 'State_Oregon'].var())

A very low variance means that the column State_Oregon completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/usr/local/lib/python3.12/dist-packages/lifelines/fitters/coxph_fitter.py:1614: ConvergenceWarning: Newton-Raphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence

<lifelines.CoxPHFitter: fitted with 5000 total observations, 4657 right-censored observations>

In [ ]:
X_death = death_data.drop(columns=["Medcal_Record_Number", "T", "E"])
times_death = np.linspace(30, 365*30, 400)   # up to 30 years

surv_death_funcs = cph_death.predict_survival_function(X_death, times=times_death)
surv_death_mat = surv_death_funcs.to_numpy().T  # (n, times)

# expected survival days
expected_surv_days = np.trapz(surv_death_mat, times_death, axis=1)

def surv_death_at(h):
    idx = np.argmin(np.abs(times_death - h))
    return surv_death_mat[:, idx]

surv_5yr  = surv_death_at(365*5)
surv_10yr = surv_death_at(365*10)
surv_20yr = surv_death_at(365*20)
surv_30yr = surv_death_at(365*30)

death_pred = pd.DataFrame({
    "Medcal_Record_Number": death_data["Medcal_Record_Number"].values,
    "Predicted_Survival_Days": expected_surv_days.round(0),
    "Predicted_Death_Risk_5yr":  (1 - surv_5yr).round(2),
    "Predicted_Death_Risk_10yr": (1 - surv_10yr).round(2),
    "Predicted_Death_Risk_20yr": (1 - surv_20yr).round(2),
    "Predicted_Death_Risk_30yr": (1 - surv_30yr).round(2),
})


/tmp/ipython-input-3077623342.py:8: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  expected_surv_days = np.trapz(surv_death_mat, times_death, axis=1)


In [ ]:
X_rec = rec_data.drop(columns=["Medcal_Record_Number", "T", "E"])
times_rec = np.linspace(30, 365*5, 200)  # up to 5 years

surv_rec_funcs = cph_rec.predict_survival_function(X_rec, times=times_rec)
surv_rec_mat = surv_rec_funcs.to_numpy().T

def surv_rec_at(h):
    idx = np.argmin(np.abs(times_rec - h))
    return surv_rec_mat[:, idx]

surv_rec_6mo = surv_rec_at(180)
surv_rec_1yr = surv_rec_at(365)

rec_pred = pd.DataFrame({
    "Medcal_Record_Number": rec_data["Medcal_Record_Number"].values,
    "Predicted_Recovery_Prob_6mo": (1 - surv_rec_6mo).round(2),
    "Predicted_Recovery_Prob_1yr": (1 - surv_rec_1yr).round(2),
})


In [ ]:
# Merge death + recovery predictions
pred = death_pred.merge(rec_pred, on="Medcal_Record_Number", how="left")

# Attach some covariates for interpretation (from original patients/env)
covars_to_attach = [
    "Medcal_Record_Number", "Age", "Gender", "State", "BMI",
    "Smoking_Status", "Pollution_Index", "Adjusted_Pollution",
    "Env_Health_Burden"
]

# use death_df_s (aligned with death_data IDs)
attach_df = death_df_s[covars_to_attach].drop_duplicates("Medcal_Record_Number")

final_out = pred.merge(attach_df, on="Medcal_Record_Number", how="left")

final_out.head()


,Medcal_Record_Number,Predicted_Survival_Days,Predicted_Death_Risk_5yr,Predicted_Death_Risk_10yr,Predicted_Death_Risk_20yr,Predicted_Death_Risk_30yr,Predicted_Recovery_Prob_6mo,Predicted_Recovery_Prob_1yr,Age,Gender,State,BMI,Smoking_Status,Pollution_Index,Adjusted_Pollution,Env_Health_Burden
0,P305837579,10668.0,0.0,0.01,0.03,0.06,0.06,0.08,51,Other,Massachusetts,29.9,Never,0.070602,0.013212,0.043
1,P662395701,10511.0,0.0,0.02,0.05,0.10,0.06,0.08,8,Other,California,39.3,Never,0.061186,0.017774,0.087
2,P173232193,10369.0,0.0,0.03,0.07,0.14,0.06,0.08,42,Female,Arizona,36.8,Never,0.016713,0.004687,0.075
3,P245798166,10411.0,0.0,0.03,0.07,0.13,0.06,0.07,34,Male,New York,21.6,Former,0.001701,0.000420,0.059
4,P938508314,10304.0,0.0,0.03,0.08,0.16,0.06,0.07,9,Other,Texas,25.5,Never,0.033341,0.012090,0.112


In [ ]:
out_xlsx = "/content/Predictive_Survival_Recovery_Final.xlsx"
out_csv  = "/content/Predictive_Survival_Recovery_Final.csv"

final_out.to_excel(out_xlsx, index=False)
final_out.to_csv(out_csv, index=False)

print("Saved:")
print(out_xlsx)
print(out_csv)


Saved:
/content/Predictive_Survival_Recovery_Final.xlsx
/content/Predictive_Survival_Recovery_Final.csv


In [ ]:
!pip install scikit-survival --quiet

from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.5 MB/s eta 0:00:00


In [ ]:
death_data = prepare_survival_df(death_df_s, "duration_days_death", "event_death")


In [ ]:
def prepare_survival_df(df, duration_col, event_col):
    data = df[["Medcal_Record_Number", duration_col, event_col] + feature_cols].copy()

    # numeric
    for c in num_cols:
        data[c] = data[c].astype(float)
        data[c] = data[c].fillna(data[c].median())

    # categorical
    for c in cat_cols:
        data[c] = data[c].fillna("Unknown")

    data[duration_col] = data[duration_col].fillna(data[duration_col].median())
    data[event_col] = data[event_col].fillna(0).astype(int)

    data_dum = pd.get_dummies(data, columns=cat_cols, drop_first=True)
    data_dum = data_dum.replace([np.inf, -np.inf], np.nan)
    data_dum = data_dum.dropna()

    data_dum = data_dum.rename(columns={duration_col: "T", event_col: "E"})
    return data_dum

death_data = prepare_survival_df(death_df_s, "duration_days_death", "event_death")

print(death_data.shape)
death_data.head()


(5000, 40)


,Medcal_Record_Number,T,E,Age,BMI,Medicaid_eligible,Living_Alone,Opioid_Addiction,Pollution_Index,Adjusted_Pollution,...,State_Illinois,State_Massachusetts,State_Michigan,State_New York,State_Oregon,State_Texas,State_Washington,Health_Access_Category_Low Access,Health_Access_Category_Moderate Access,Health_Access_Category_Unknown
89011,P305837579,1,0,51.0,29.9,1.0,0.0,0.0,0.070602,0.013212,...,False,True,False,False,False,False,False,False,False,False
89834,P662395701,1,0,8.0,39.3,1.0,1.0,0.0,0.061186,0.017774,...,False,False,False,False,False,False,False,False,True,False
87343,P173232193,1,0,42.0,36.8,0.0,0.0,0.0,0.016713,0.004687,...,False,False,False,False,False,False,False,False,True,False
6046,P245798166,1,0,34.0,21.6,0.0,1.0,0.0,0.001701,0.000420,...,False,False,False,True,False,False,False,False,True,False
14186,P938508314,1,0,9.0,25.5,0.0,0.0,0.0,0.033341,0.012090,...,False,False,False,False,False,True,False,True,False,False


In [ ]:
# Features and labels
X_rsf = death_data.drop(columns=["Medcal_Record_Number", "T", "E"])
y_rsf = Surv.from_arrays(
    event=death_data["E"].astype(bool),
    time=death_data["T"].astype(float)
)

X_train, X_test, y_train, y_test = train_test_split(
    X_rsf, y_rsf, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape


((4000, 37), (1000, 37))

In [ ]:
rsf = RandomSurvivalForest(
    n_estimators=200,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

rsf.fit(X_train, y_train)


RandomSurvivalForest(min_samples_leaf=5, min_samples_split=10, n_estimators=200,
                     n_jobs=-1, random_state=42)

In [ ]:
# risk scores (higher = higher risk of death)
risk_pred_test = rsf.predict(X_test)

c_index = concordance_index_censored(
    y_test["event"],
    y_test["time"],
    risk_pred_test
)[0]

print(f"RSF C-index: {c_index:.3f}")


RSF C-index: 0.517


In [ ]:
import numpy as np
import pandas as pd

# 1) Predict survival functions for ALL rows in death_data
# This returns a list of StepFunction objects, one per patient
surv_funcs = rsf.predict_survival_function(X_rsf)

# Use the time grid from the first patient (same for all)
times = surv_funcs[0].x  # array of time points

# Stack survival probabilities into an (n_samples, n_times) array
surv_array = np.vstack([f.y for f in surv_funcs])

# 2) Expected survival days ≈ area under survival curve
expected_surv_days_rsf = np.trapz(surv_array, times, axis=1)

# Helper: survival at a specific horizon h (days)
def surv_at(h):
    idx = np.argmin(np.abs(times - h))
    return surv_array[:, idx]

surv_5yr  = surv_at(365*5)
surv_10yr = surv_at(365*10)
surv_20yr = surv_at(365*20)
surv_30yr = surv_at(365*30)

# 3) Build prediction dataframe
death_pred_rsf = pd.DataFrame({
    "Medcal_Record_Number": death_data["Medcal_Record_Number"].values,
    "RSF_Predicted_Survival_Days": expected_surv_days_rsf.round(0),
    "RSF_Death_Risk_5yr":  (1 - surv_5yr).round(2),
    "RSF_Death_Risk_10yr": (1 - surv_10yr).round(2),
    "RSF_Death_Risk_20yr": (1 - surv_20yr).round(2),
    "RSF_Death_Risk_30yr": (1 - surv_30yr).round(2),
})

death_pred_rsf.head()



/tmp/ipython-input-2432303416.py:15: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  expected_surv_days_rsf = np.trapz(surv_array, times, axis=1)


,Medcal_Record_Number,RSF_Predicted_Survival_Days,RSF_Death_Risk_5yr,RSF_Death_Risk_10yr,RSF_Death_Risk_20yr,RSF_Death_Risk_30yr
0,P305837579,21874.0,0.00,0.03,0.09,0.16
1,P662395701,23854.0,0.00,0.01,0.01,0.05
2,P173232193,23176.0,0.00,0.00,0.01,0.09
3,P245798166,20146.0,0.08,0.09,0.10,0.19
4,P938508314,18986.0,0.00,0.12,0.22,0.30


In [ ]:
covars_to_attach = [
    "Medcal_Record_Number", "Age", "Gender", "State", "BMI",
    "Smoking_Status", "Pollution_Index", "Adjusted_Pollution",
    "Env_Health_Burden"
]

attach_df = death_df_s[covars_to_attach].drop_duplicates("Medcal_Record_Number")

rsf_final = death_pred_rsf.merge(attach_df, on="Medcal_Record_Number", how="left")

rsf_xlsx = "/content/RSF_Predictive_Survival_Death.xlsx"
rsf_csv  = "/content/RSF_Predictive_Survival_Death.csv"

rsf_final.to_excel(rsf_xlsx, index=False)
rsf_final.to_csv(rsf_csv, index=False)

print("Saved:")
print(rsf_xlsx)
print(rsf_csv)


Saved:
/content/RSF_Predictive_Survival_Death.xlsx
/content/RSF_Predictive_Survival_Death.csv
